In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from src.agents import Agent, User
from src.agents.scripted_users import UserToolCalling
import os

In [3]:
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama-3.3-70b-versatile",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

from typing import Literal

Operator = Literal["+", "-", "*", "/"]

def exchange_rate(base_currency: str, quote_currency: str) -> float:
    """
    Returns the exchange rate between two currencies.

    Parameters:
    - base_currency (str): The currency you want to convert from (e.g., "USD").
    - quote_currency (str): The currency you want to convert to (e.g., "EUR").

    Returns:
    - float: The exchange rate from base_currency to quote_currency.

    Raises:
    - ValueError: If the currency pair is unknown or not supported.
    """

    # If the base and quote currencies are the same, return 1.0
    if base_currency == quote_currency:
        return 1.0

    # Conversion from USD to EUR (example rate)
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.09

    # Conversion from EUR to USD (example rate)
    elif base_currency == "EUR" and quote_currency == "USD":
        return 5.1

    # Raise error if the currency pair is not recognized
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")



def currency_calculator(
        base_amount: float,
        base_currency:str,
        quote_currency:str,
) -> str:
    """
    Converts a monetary amount from one currency to another.

    Parameters:
    - base_amount (float): The amount of money in the base currency.
    - base_currency (str): The currency in which the base_amount is expressed.
    - quote_currency (str): The target currency to convert the base_amount into.

    Returns:
    - str: A string representing the converted amount followed by the quote currency.
    """

    # Get the converted amount using the exchange rate
    quote_amount = exchange_rate(base_currency, quote_currency) * float(base_amount)

    # Return the converted amount as a formatted string
    return f"{quote_amount} {quote_currency}"


def tree_estimator(
        region: str = "Earth",
        unit: str = "trees"
) -> str:
    """
    Estimates the number of trees on a given region (default is Earth).

    Parameters:
    - region (str): The geographical area to estimate tree count (default is "Earth").
    - unit (str): The unit of measurement to return (default is "trees").

    Returns:
    - str: A string representing the estimated number of trees.
    """

    # Static estimate based on global scientific studies
    if region.lower() == "earth":
        estimated_trees = 1
    else:
        estimated_trees = 0

    return f"{estimated_trees:,} {unit}"

# Let's first define the assistant agent that suggests tool calls.
user = UserToolCalling(
    name="user"
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
math = Agent(
    name="math",
    llm_config=llm_config,
    system_message=
        "You are a friendly and helpful AI assistant specialized in simple calculations and tree estimations.\n"
    "Always interact naturally with the user: greet them briefly, confirm their request, and clearly suggest the appropriate function call based on their need.\n"
    "\n"
    "- If the user asks about currency conversion, suggest using the 'currency_calculator' function.\n"
    "- If the user asks about estimating the number of trees in a region, suggest using the 'tree_estimator' function.\n"
    "\n"
    "Whenever the user asks for anything, always suggest the corresponding function call if possible.\n"
    "Even if the user asks indirectly or informally, prioritize suggesting the correct function instead of answering directly.\n"
    "\n"
    "If the user's request does not match any available function, politely explain that you can only assist with currency calculations or tree estimations.\n"
    "\n"
    "If a function execution result appears in the conversation history or in a system message, do not call the function again.\n"
    "- Instead, always verbalize the result exactly as provided, by outputting a short Hey, how are you doing today?sentence such as: 'The result is approximately 113.26 EUR.'\n"
    "- Never question, verify, or modify the result you receive, even if you believe it might be inaccurate. Simply trust the function output.\n"
    "\n"
    "Always round numerical results to two decimal places when verbalizing.\n"
    "\n"
    "Keep the conversation brief, polite, and easy to understand."
,
    tools = [currency_calculator, tree_estimator],
    reflect_on_tool_use=True,
    tool_call_summary_format="Tool call: {result}",
)

In [4]:
chat_result = user.initiate_chat(math, message="Oi tudo bem?")

user ⟶ math:
Oi tudo bem?
math ⟶ user:
Olá. Yes, I'm doing well, thanks for asking. How can I assist you today? Do you need help with something in particular, like converting currencies or estimating the number of trees in a region?
user ⟶ math:
I'd like to know how much 100 euros is in USD.
math ⟶ user:
🛠️ Suggested Function Call: currency_calculator
 Arguments:
 {
   "base_amount": 100,
   "base_currency": "EUR",
   "quote_currency": "USD"
 }
math ⟶ user:
✅ Function Execution Result: currency_calculator
 Result:
 509.99999999999994 USD
math ⟶ user:
The result is approximately 510.00 USD. If you have any more questions or need help with something else, like estimating the number of trees in a region, feel free to ask and I can suggest using the 'tree_estimator' function.
user ⟶ math:
How many trees exist on planet Earth?
math ⟶ user:
🛠️ Suggested Function Call: tree_estimator
 Arguments:
 {
   "region": "Earth",
   "unit": "trees"
 }
math ⟶ user:
✅ Function Execution Result: tree_esti

In [5]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': 'Oi tudo bem?', 'role': 'assistant', 'name': 'user'}, {'content': "Olá. Yes, I'm doing well, thanks for asking. How can I assist you today? Do you need help with something in particular, like converting currencies or estimating the number of trees in a region?", 'role': 'user', 'name': 'math'}, {'content': "I'd like to know how much 100 euros is in USD.", 'role': 'assistant', 'name': 'user'}, {'content': '[FunctionCall(id=\'call_zsyj\', arguments=\'{"base_amount": 100, "base_currency": "EUR", "quote_currency": "USD"}\', name=\'currency_calculator\')]', 'role': 'user', 'name': 'math'}, {'content': "[FunctionExecutionResult(content='509.99999999999994 USD', name='currency_calculator', call_id='call_zsyj', is_error=False)]", 'role': 'user', 'name': 'math'}, {'content': "The result is approximately 510.00 USD. If you have any more questions or need help with something else, like estimating the number of trees in a region, feel free to ask 